In [2]:
from utilities import *
import numpy as np
import cv2
import os
import re
import importlib



In [3]:
input_dir=r"Z:\lab\NSF forelimb project\Phil_lab\C-arm\In"
all_avis = scanDir(input_dir,"avi")
data_avis = [path for path in all_avis if not re.search("cube", path)]
nocal_avis = [path for path in data_avis if not re.search("cal", path)]
actual_avis = [path for path in nocal_avis if not re.search("preview", path)]
raw_avis = [path for path in actual_avis if not re.search("robo", path)]


# video_dict = {}
# video_dict['A'], video_dict['B'] = scanDir(input_dir, "avi")
# codec = 0
# #substitute 0 in call to videowriter for uncompressed, otherwise cv2.VideoWriter_fourcc(*'mp4v')
# ##https://www.microsoft.com/en-us/p/hevc-video-extensions-from-device-manufacturer/9n4wgh0z6vhq?activetab=pivot:overviewtab


In [4]:
trial_dirs = list(set([os.path.dirname(path) for path in raw_avis]))
trial_dict = {}
for directory in trial_dirs:
    trial_dir, trial_name = os.path.split(directory)
    day_dir, day_name = os.path.split(trial_dir)
    day = day_name.split('_',1)[0]
    uid = day + '_' + trial_name
    avis = [file for file in raw_avis if re.search(day_name, file) and re.search(trial_name,file)]
#     print(avis)
    cam1 = [os.path.basename(file) for file in avis if re.search('1', os.path.basename(file)[-5:])]
    cam2 = [os.path.basename(file) for file in avis if re.search('2', os.path.basename(file)[-5:])]
    if len(cam1)!=1:
        print(directory)
        print(cam1)
    if len(cam2)!=1:
        print(directory)
        print(cam2)
    trial_dict[uid] = {"path":directory, "c1":cam1, "c2":cam2}


Z:\lab\NSF forelimb project\Phil_lab\C-arm\In\Oct7_LaiWright\19_t2_fullstride_B+_HINDLIMB
[]
Z:\lab\NSF forelimb project\Phil_lab\C-arm\In\Oct7_LaiWright\19_t2_fullstride_B+_HINDLIMB
[]


In [ ]:
done = []
for trial in list(trial_dict.keys()):
    if trial_dict[trial]['c1'] and trial_dict[trial]['c2'] :
        print('creating tandem preview for '+trial_dict[trial]['path'])
        tandemPreviews(trial_dict[trial])
        done.append(trial)
    else:
        print('skipping '+trial_dict[trial]['path']+' unpaired camera views')

In [46]:
test_trials = {'tegu_lo':'Oct7_8_t4_fullstride_A', 
               'tegu_hi':'Nov1_5_t1_squeeze_A+',
               'possum_lo':'Nov6_12_p1_squeezelow_A',
               'possum_hi':'Oct14_9_p4_fullstride_a+',
              }

In [143]:
# foreach: laplacian, 1-5x kernel
def laplacianKernelSearch(video_dict):
    fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
    capA = cv2.VideoCapture(os.path.join(video_dict['path'],video_dict['c1'][0]))
    capB = cv2.VideoCapture(os.path.join(video_dict['path'],video_dict['c2'][0]))
    frame_index = 0
    frame_width = int(capA.get(3))
    frame_height = int(capA.get(4))
    frame_rate = round(capA.get(5),2)
    input_name =os.path.splitext(video_dict['c1'][0])
    output_name = os.path.join(video_dict['path'],(input_name[0][:-1]+'_laplacian'+'.mp4'))
    
    frameA_yStart = math.floor(frame_height*.1)
    frameA_yEnd = math.floor(frame_height*.8)
    frameB_yStart = math.floor(frame_height*.3)
    frameB_yEnd = math.floor(frame_height)
    
    output_dims = [frame_width*2, frameA_yEnd-frameA_yStart+frameB_yEnd-frameB_yStart]
    out = cv2.VideoWriter(output_name,
                         fourcc,
                         frame_rate,(output_dims[0], output_dims[1]))
    while(capA.isOpened()):
        retA, frameA = capA.read()
        retB, frameB = capB.read()
        frame_index += 1
        if retA == True:
            frameArotate = imutils.rotate(frameA, 128)
            frameAcrop = frameArotate[frameA_yStart:frameA_yEnd,:]
            frameBcrop = frameB[frameB_yStart:frameB_yEnd,:]
            concatenated = cv2.vconcat([frameAcrop, frameBcrop])
            preview = cv2.resize(concatenated, (math.floor(frame_width/2), (math.floor(output_dims[1]/2))))
            
            gray_img = cv2.cvtColor(preview, cv2.COLOR_BGR2GRAY)
            clahe1 = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(32,32))
            clahe2 = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(40,40))
            cl1 = clahe1.apply(gray_img)
            cl2 = clahe2.apply(gray_img)

#             blur_img1 = cv2.GaussianBlur(gray_img, (1, 1), 0)
#             laplacian1 = cv2.Laplacian(blur_img1, cv2.CV_64F)
#             blur_img3 = cv2.GaussianBlur(gray_img, (3, 3), 0)
#             laplacian3 = cv2.Laplacian(blur_img3, cv2.CV_64F)
#             blur_img5 = cv2.GaussianBlur(gray_img, (5, 5), 0)
#             laplacian5 = cv2.Laplacian(blur_img5, cv2.CV_64F)
#             edges = cv2.Canny(cl1, 40, 120, 3, L2gradient=True)
            combined = cv2.hconcat([gray_img,cl1,cl2])
#             labeled1 = cv2.putText(laplacian1, '1x1 laplacian', (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255))
#             labeled3 = cv2.putText(laplacian3, '3x3 laplacian', (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255))
#             labeled5 = cv2.putText(laplacian5, '5x5 laplacian', (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255))
#             both = cv2.hconcat([labeled1, labeled3, labeled5])

#             labeled = cv2.putText(preview, video_dict['path'], (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255))
#             labeled = cv2.putText(preview, 'FRAME '+str(frame_index), (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255))
#             labeled = cv2.putText(preview, 'DIMS '+str(preview.shape), (20,60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255))
#             cv2.cvtColor(labeled, cv2.COLOR_BGR2GRAY)  
            cv2.imshow('test',combined)
#             out.write(both)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                frame_index = 0
                break
        else:
            frame_index = 0
            break
    capA.release()
    capB.release()
    out.release()
    cv2.destroyAllWindows()

# def mergeRGB(video_dict, codec, mode):
#     capA = cv2.VideoCapture(video_dict['A'])
#     capB = cv2.VideoCapture(video_dict['B'])
#     frame_width = int(capA.get(3))
#     frame_height = int(capA.get(4))
#     frame_rate = round(capA.get(5),2)
#     input_name =os.path.splitext(os.path.basename(video_dict['A']))
#     output_name = mode+"_RGBMerge_"+input_name[0][:-4]+input_name[1]
#     out = cv2.VideoWriter(output_name,
#                          codec,
#                          frame_rate,(frame_width, frame_height))
#     while(capA.isOpened()):
#         retA, frameA = capA.read()
#         retB, frameB = capB.read()
#         if retA == True:
#             ## give frames an alpha channel to prepare for blending; blend_modes requires 32bit
#             frameA = cv2.cvtColor(frameA, cv2.COLOR_BGR2BGRA,4).astype(np.float32)
#             frameB = cv2.cvtColor(frameB, cv2.COLOR_BGR2BGRA,4).astype(np.float32)
#             if mode == "difference":
#                 extraChannel = blend_modes.difference(frameA,frameB,1)
#             elif mode == "multiply":
#                 extraChannel = blend_modes.multiply(frameA,frameB,1)
#             else:
#                 extraChannel = np.zeros((frame_width, frame_height,3),np.uint8)
#                 extraChannel = cv2.cvtColor(extraChannel, cv2.COLOR_BGR2BGRA,4).astype(np.float32)

#             ## get rid of alpha channel in preparation for converting back to grayscale; opencv prefers 8bit
#             frameA = cv2.cvtColor(frameA, cv2.COLOR_BGRA2BGR).astype(np.uint8)  
#             frameB = cv2.cvtColor(frameB, cv2.COLOR_BGRA2BGR).astype(np.uint8)  
#             extraChannel = cv2.cvtColor(extraChannel, cv2.COLOR_BGRA2BGR).astype(np.uint8)  

#             ## convert to grayscale so we can merge into 3-channel image
#             frameA = cv2.cvtColor(frameA, cv2.COLOR_BGR2GRAY)  
#             frameB = cv2.cvtColor(frameB, cv2.COLOR_BGR2GRAY)  
#             extraChannel = cv2.cvtColor(extraChannel, cv2.COLOR_BGR2GRAY)  

#             ## merge, show and write                  
#             merged = cv2.merge((extraChannel, frameB, frameA))
#             cv2.imshow('merged',merged)
#             out.write(merged)
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break
#         else:
#             break
#     capA.release()
#     capB.release()
#     out.release()
#     cv2.destroyAllWindows()
#     print("done!")cc

In [145]:
laplacianKernelSearch(trial_dict[test_trials['tegu_lo']])